In [ ]:
! pip install automathon

  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.21
    Uninstalling graphviz-0.21:
      Successfully uninstalled graphviz-0.21


In [ ]:
# Install automathon first: pip install automathon

from automathon import DFA

# DFA definition
# States: q0 (start), q1 (valid), q_dead (invalid)
# Alphabet: lowercase letters a-z
# Transitions:
#   q0 --lowercase--> q1
#   q1 --lowercase--> q1
# Any other input -> q_dead

states = {'q0', 'q1'}
input_symbols = set('abcdefghijklmnopqrstuvwxyz')
transitions = {
    'q0': {ch: 'q1' for ch in input_symbols},
    'q1': {ch: 'q1' for ch in input_symbols}
}

# Start and final states
initial_state = 'q0'
final_states = {'q1'}

# Create DFA
dfa = DFA(states, input_symbols, transitions, initial_state, final_states)

# Function to check a word
def check_word(word):
    # First, reject if any char not lowercase a-z
    if not word or any(ch not in input_symbols for ch in word):
        return "Not Accepted"
    # Process in DFA
    if dfa.accept(word):
        return "Accepted"
    return "Not Accepted"

# Test cases
words = ["cat", "dog", "a", "zebra", "dog1", "1dog", "DogHouse", "Dog_house", " cats"]
for w in words:
    print(f"{w!r} -> {check_word(w)}")

# Visualization
dfa.view("dfa_diagram.png")  # Saves DFA diagram as PNG

'cat' -> Accepted
'dog' -> Accepted
'a' -> Accepted
'zebra' -> Accepted
'dog1' -> Not Accepted
'1dog' -> Not Accepted
'DogHouse' -> Not Accepted
'Dog_house' -> Not Accepted
' cats' -> Not Accepted


In [ ]:
import string

# Load all nouns from the brown_nouns.txt file
with open("/content/brown_nouns.txt") as f:
    nouns = [w.strip() for w in f if w.strip()]

# Utility function: check if letter is vowel
def is_vowel(ch):
    return ch in "aeiou"

# FST simulation
def analyze_word(word):
    # Reject if contains characters outside lowercase letters
    if not word.isalpha() or not word.islower():
        return "Invalid Word"

    # Singular: no plural suffix
    if not word.endswith('s'):
        return f"{word}+N+SG"

    # Plural candidates:
    if word.endswith("ies"):
        # Rule: Y replacement: consonant + y -> ies
        if len(word) >= 4 and not is_vowel(word[-4]):
            root = word[:-3] + "y"
            return f"{root}+N+PL"
        else:
            return "Invalid Word"

    elif word.endswith("es"):
        # Rule: E insertion: after s, z, x, ch, sh
        root = word[:-2]
        if root.endswith(("s", "x", "z", "ch", "sh")):
            return f"{root}+N+PL"
        else:
            return "Invalid Word"

    else:
        # Rule: Simple S addition
        root = word[:-1]
        # Reject if it should have used 'es' or 'ies'
        if root.endswith(("s", "x", "z", "ch", "sh")):
            return "Invalid Word"
        if len(root) >= 2 and root.endswith("y") and not is_vowel(root[-2]):
            return "Invalid Word"
        return f"{root}+N+PL"

# Test with given examples
examples = ["cat", "dog", "a", "zebra", "foxes", "fox", "foxs",
            "tries", "try", "trys", "toies", "bags"]

for w in examples:
    print(f"{w} -> {analyze_word(w)}")

# Process all nouns from the corpus
results = {w: analyze_word(w) for w in nouns}

# Save output for inspection
with open("noun_analysis.txt", "w") as f:
    for w, analysis in results.items():
        f.write(f"{w} = {analysis}\n")

print("\nAnalysis complete. Results saved to noun_analysis.txt")


cat -> cat+N+SG
dog -> dog+N+SG
a -> a+N+SG
zebra -> zebra+N+SG
foxes -> fox+N+PL
fox -> fox+N+SG
foxs -> Invalid Word
tries -> try+N+PL
try -> try+N+SG
trys -> Invalid Word
toies -> Invalid Word
bags -> bag+N+PL

Analysis complete. Results saved to noun_analysis.txt


In [4]:
from pathlib import Path
import sys
import string

ALPHABET = "abcdefghijklmnopqrstuvwxyz"

class FST:
    """
    Deterministic FST implemented with an explicit transition table:
    transitions: dict[(state, char)] -> (next_state, out_str)

    For efficiency and clarity we implement buffer states to defer output of
    characters that might be part of a plural suffix (s, x, z, c, i).
    """
    def __init__(self, lexicon=None):
        self.transitions = {}
        self.start = 'q0'
        self.lexicon = lexicon or set()
        self._build_transitions()
        # acceptance flush/tag mapping for states that are final when input ends there
        self.accept_map = {
            'q1': ('', '+N+SG'),             # normal final state -> singular
            # q_buf_s is ambiguous -> handle specially via lexicon
            'q_se': ('se', '+N+SG'),
            'q_sh': ('sh', '+N+SG'),
            'q_buf_x': ('x', '+N+SG'),
            'q_xe': ('xe', '+N+SG'),
            'q_buf_z': ('z', '+N+SG'),
            'q_ze': ('ze', '+N+SG'),
            'q_buf_c': ('c', '+N+SG'),
            'q_ch': ('ch', '+N+SG'),
            'q_che': ('che', '+N+SG'),
            'q_buf_i': ('i', '+N+SG'),
            'q_ie': ('ie', '+N+SG'),
            'qF': ('', ''),  # already produced tag in transition output
        }

    def _add(self, state, ch, next_state, out):
        self.transitions[(state, ch)] = (next_state, out)

    def _build_transitions(self):
        # Start: first letter -> q1 (emit that letter)
        for c in ALPHABET:
            self._add('q0', c, 'q1', c)

        # In q1 (confirmed-stem state) ordinary letters simply emit and stay in q1
        specials = {'s', 'x', 'z', 'c', 'i'}  # letters that could begin suffix sequences
        for c in ALPHABET:
            if c not in specials:
                self._add('q1', c, 'q1', c)

        # For "potential suffix beginning" letters we move to buffer states (no output yet)
        self._add('q1', 's', 'q_buf_s', '')   # 's' might be suffix or stem-final s
        self._add('q1', 'x', 'q_buf_x', '')   # 'x' might be followed by 'e'+'s' -> 'xes'
        self._add('q1', 'z', 'q_buf_z', '')   # similarly 'z' -> 'zes'
        self._add('q1', 'c', 'q_buf_c', '')   # 'ch' detection
        self._add('q1', 'i', 'q_buf_i', '')   # detect 'ies' -> y + PL

        # q_buf_s logic: we saw 's' but didn't emit yet.
        # If next is 'e' -> could become '...ses' -> go q_se
        self._add('q_buf_s', 'e', 'q_se', '')
        # If next is 'h' -> could be 'sh' -> go q_sh
        self._add('q_buf_s', 'h', 'q_sh', '')
        # Otherwise: 's' wasn't a suffix; flush 's' plus the next letter
        for c in ALPHABET:
            if c not in {'e', 'h'}:
                # we flush the buffered 's' plus the current char and continue as q1
                # (out string contains both characters consumed)
                self._add('q_buf_s', c, 'q1', 's' + c)

        # q_se: we've seen 's' then 'e' (buffered). If next 's' -> plural for roots that end with s etc
        # Accepting transition produces the stem's final char (s) + tag
        self._add('q_se', 's', 'qF', 's+N+PL')  # outputs 's+N+PL'
        for c in ALPHABET:
            if c != 's':
                self._add('q_se', c, 'q1', 'se' + c)

        # q_sh: seen 's' then 'h'
        self._add('q_sh', 'e', 'q_she', '')
        for c in ALPHABET:
            if c != 'e':
                self._add('q_sh', c, 'q1', 'sh' + c)
        # q_she handles 'sh' + 'e' then possibly 's' => 'shes'
        self._add('q_she', 's', 'qF', 'sh+N+PL')
        for c in ALPHABET:
            if c != 's':
                self._add('q_she', c, 'q1', 'she' + c)

        # q_buf_x: try 'xe' then 's' => 'xes' plural; disallow immediate 's' (e.g., 'foxs' invalid)
        self._add('q_buf_x', 'e', 'q_xe', '')
        for c in ALPHABET:
            if c not in {'e', 's'}:  # do not allow 'x' followed directly by 's' -> invalid
                self._add('q_buf_x', c, 'q1', 'x' + c)
        self._add('q_xe', 's', 'qF', 'x+N+PL')
        for c in ALPHABET:
            if c != 's':
                self._add('q_xe', c, 'q1', 'xe' + c)

        # q_buf_z -> 'ze' -> 'zes'
        self._add('q_buf_z', 'e', 'q_ze', '')
        for c in ALPHABET:
            if c not in {'e', 's'}:
                self._add('q_buf_z', c, 'q1', 'z' + c)
        self._add('q_ze', 's', 'qF', 'z+N+PL')
        for c in ALPHABET:
            if c != 's':
                self._add('q_ze', c, 'q1', 'ze' + c)

        # q_buf_c -> maybe 'ch'
        self._add('q_buf_c', 'h', 'q_ch', '')
        for c in ALPHABET:
            if c not in {'h', 's'}:
                self._add('q_buf_c', c, 'q1', 'c' + c)
        self._add('q_ch', 'e', 'q_che', '')
        for c in ALPHABET:
            if c != 'e':
                self._add('q_ch', c, 'q1', 'ch' + c)
        self._add('q_che', 's', 'qF', 'ch+N+PL')
        for c in ALPHABET:
            if c != 's':
                self._add('q_che', c, 'q1', 'che' + c)

        # q_buf_i -> possibly part of 'ies' plural
        self._add('q_buf_i', 'e', 'q_ie', '')
        for c in ALPHABET:
            if c != 'e':
                self._add('q_buf_i', c, 'q1', 'i' + c)
        self._add('q_ie', 's', 'qF', 'y+N+PL')  # i e s -> y + PL
        for c in ALPHABET:
            if c != 's':
                self._add('q_ie', c, 'q1', 'ie' + c)

        # qF is final after successful plural recognition (tag is already in output buffer)

    def analyze(self, token):
        """
        Analyze single token -> returns "stem+N+SG|PL" or "Invalid Word".
        We lowercase input and only accept pure alphabetic tokens here (others -> Invalid Word).
        Uses lexicon to disambiguate trailing single 's' cases.
        """
        w = token.lower()
        if not w.isalpha():
            return "Invalid Word"

        state = self.start
        out = ""   # built output (stem pieces + sometimes tags when immediate)
        for ch in w:
            trans = self.transitions.get((state, ch))
            if trans is None:
                return "Invalid Word"
            next_state, out_str = trans
            out += out_str
            state = next_state

        # If transitions produced qF it's already tagged (e.g. +N+PL included)
        if state == 'qF':
            return out

        # Special-case ambiguous buffered 's' final: need lexicon disambiguation
        if state == 'q_buf_s':
            base = w[:-1]
            # If base exists in lexicon, it is probably plural (bag->bags). But if the base
            # ends with letters that require 'es' (x,z,s,ch,sh) then a bare '...s' form should be invalid.
            if base in self.lexicon:
                bad_endings = ('s', 'z', 'x')
                if base.endswith(bad_endings) or base.endswith('ch') or base.endswith('sh'):
                    # e.g., base 'fox' -> 'foxs' is invalid, expect 'foxes'
                    return "Invalid Word"
                return out + '+N+PL'   # out already contains base (we suppressed 's' earlier)
            else:
                # If base not in lexicon, treat the final 's' as part of the stem (singular)
                return out + 's' + '+N+SG'

        # If other accepting states:
        acc = self.accept_map.get(state)
        if acc is not None:
            flush, tag = acc
            return out + flush + tag

        # Not an accepting state -> fallback analysis using lexicon-based suffix checks
        # This is used as a recovery: check if orthographic suffix patterns map to valid stems.
        # (It keeps the transducer deterministic for common cases while using lexicon to
        #  resolve remaining ambiguities.)
        return self._lexicon_suffix_fallback(w, out)

    def _lexicon_suffix_fallback(self, w, out_before):
        """
        If the FST failed to accept, try lexicon-driven heuristics:
         - w.endswith('ies') -> base = w[:-3]+'y'
         - w.endswith('es')  -> base = w[:-2]
         - w.endswith('s')   -> base = w[:-1] (but reject if base endswith x,z,s,ch,sh)
         - else if w in lexicon => singular
         - else invalid
        """
        if w.endswith('ies'):
            base = w[:-3] + 'y'
            if base in self.lexicon:
                return base + '+N+PL'
        if w.endswith('es'):
            base = w[:-2]
            if base in self.lexicon:
                return base + '+N+PL'
        if w.endswith('s'):
            base = w[:-1]
            if base in self.lexicon:
                bad_endings = ('s', 'z', 'x')
                if base.endswith(bad_endings) or base.endswith('ch') or base.endswith('sh'):
                    return "Invalid Word"
                return base + '+N+PL'
        if w in self.lexicon:
            return w + '+N+SG'
        return "Invalid Word"


def main(input_path='brown_nouns.txt', output_path='fst_results.txt'):
    p = Path(input_path)

    # load lexicon: all pure alphabetic tokens (lowercased)
    lexicon = set()
    tokens = []
    with p.open('r', encoding='utf-8') as f:
        for line in f:
            tok = line.strip()
            if not tok:
                continue
            tok_l = tok.lower()
            tokens.append(tok_l)
            if tok_l.isalpha():
                lexicon.add(tok_l)

    print(f"Loaded {len(tokens)} tokens ({len(lexicon)} alphabetic tokens in lexicon).")

    fst = FST(lexicon=lexicon)

    out_p = Path(output_path)
    invalid_count = 0
    with out_p.open('w', encoding='utf-8') as out_f:
        for i, tok in enumerate(tokens, 1):
            analysis = fst.analyze(tok)
            out_f.write(f"{tok} = {analysis}\n")
            if analysis == "Invalid Word":
                invalid_count += 1
            # occasional progress printing for large files
            if i % 20000 == 0:
                print(f"Processed {i} tokens...")

    print(f"Done. Processed {len(tokens)} tokens. Invalid words: {invalid_count}.")
    print(f"Results written to: {out_p.resolve()}")


if __name__ == "__main__":
    main('brown_nouns.txt', 'fst_results.txt')

Loaded 202793 tokens (17342 alphabetic tokens in lexicon).
Processed 20000 tokens...
Processed 40000 tokens...
Processed 60000 tokens...
Processed 80000 tokens...
Processed 100000 tokens...
Processed 120000 tokens...
Processed 140000 tokens...
Processed 160000 tokens...
Processed 180000 tokens...
Processed 200000 tokens...
Done. Processed 202793 tokens. Invalid words: 1324.
Results written to: /content/fst_results.txt


In [ ]:
from graphviz import Digraph

# Create Graphviz Digraph
fst = Digraph("PluralFST", format="png")
fst.attr(rankdir="LR", size="8")

# States
states = ["q0", "q_s", "q_es", "q_ies", "q_root_SG", "q_root_PL", "q_invalid"]
accept_states = ["q_root_SG", "q_root_PL"]

for s in states:
    if s in accept_states:
        fst.attr("node", shape="doublecircle", style="filled", fillcolor="lightgreen")
    elif s == "q_invalid":
        fst.attr("node", shape="doublecircle", style="filled", fillcolor="red")
    else:
        fst.attr("node", shape="circle", style="filled", fillcolor="lightblue")
    fst.node(s)

# Transitions
# Start transitions
fst.edge("q0", "q_s", label="s")
fst.edge("q0", "q_root_SG", label="a..z except s / emit root+N+SG")

# q_s transitions
fst.edge("q_s", "q_es", label="e")
fst.edge("q_s", "q_root_PL", label="letter not requiring es or ies / emit root+N+PL")
fst.edge("q_s", "q_invalid", label="x,z,s,ch,sh or y preceded by consonant")

# q_es transitions
fst.edge("q_es", "q_ies", label="i")
fst.edge("q_es", "q_root_PL", label="x,z,s,ch,sh / emit root+N+PL")
fst.edge("q_es", "q_invalid", label="otherwise")

# q_ies transitions
fst.edge("q_ies", "q_root_PL", label="consonant / root ends with y")
fst.edge("q_ies", "q_invalid", label="vowel before y")

# Root processing loops
fst.edge("q_root_SG", "q_root_SG", label="a..z / accumulate root")
fst.edge("q_root_PL", "q_root_PL", label="a..z / accumulate root")

# Invalid loops
fst.edge("q_invalid", "q_invalid", label="a..z")

# Save and render to file in /mnt/data
output_path = "plural_fst"
fst.render(output_path, cleanup=True)
print(f"FST diagram saved to {output_path}.png")


FST diagram saved to plural_fst.png
